#  Setup & Imports

In [40]:

import pandas as pd
from oracle_db_manager import OracleDBManager
from dotenv import load_dotenv


# Load Cleaned Data

In [15]:
df = pd.read_csv('../data/bank_reviews_clean.csv')
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
df.head()

,review,rating,date,bank,source
0,👌👍,5,2025-06-04,Commercial Bank of Ethiopia,Google Play Store
1,very niec,5,2025-06-04,Commercial Bank of Ethiopia,Google Play Store
2,best app of finance,5,2025-06-04,Commercial Bank of Ethiopia,Google Play Store
3,yetemeta,1,2025-06-03,Commercial Bank of Ethiopia,Google Play Store
4,Engida Kebede Fetera,5,2025-06-03,Commercial Bank of Ethiopia,Google Play Store


# Load .env variables

In [28]:
load_dotenv()

True

#  -- Initialize and Connect to Oracle --

#### Create DB instance using environment variables

In [41]:
db = OracleDBManager(
    user=os.getenv("ORACLE_USER"),
    password=os.getenv("ORACLE_PASSWORD"),
    dsn=os.getenv("ORACLE_DSN")
)
# Test connection
db.connect()

 Connected to Oracle DB.


# Create Tables (Run Once Only)

In [17]:
# Run this only once, or comment out after tables are created
try:
    db.create_tables()
except Exception as e:
    print("⚠️ Tables may already exist:", e)

 Tables created.


#  -------- Insert Banks -------- 

In [18]:
bank_names = df['bank'].unique()
bank_id_map = db.insert_banks(bank_names)
bank_id_map

{'Commercial Bank of Ethiopia': 1, 'Bank of Abyssinia': 2, 'Dashen Bank': 3}

#  -------- Insert Reviews -------- 

In [19]:
db.insert_reviews(df, bank_id_map)

 Reviews inserted.


# -------- Verify in Oracle -------- 

In [20]:
db.connect()
db.cursor.execute("SELECT COUNT(*) FROM reviews")
print("Review count:", db.cursor.fetchone()[0])

 Connected to Oracle DB.
Review count: 1106


# ---- Close Connection ---- 

In [42]:
db.close()

🔌 Connection closed.
